<a href="https://colab.research.google.com/github/Alireza-Khodapanah/Similarity-of-vulnerabilities/blob/main/vulnerabilities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask-ngrok
!ngrok authtoken '2rALs1L1VhMwt9but0MYQXXYFit_2rnSZuYUGd2p4ggy6A5HW'

/bin/bash: line 1: ngrok: command not found


In [ ]:
!pip install nltk spacy sentence-transformers
!python -m spacy download en_core_web_sm
!pip install flask pyngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 42.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [1]:
import pandas as pd
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

In [2]:
file_path = "/content/vuln2.csv"

In [3]:
# Download NLTK resources
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

# Load spacy model
nlp = spacy.load("en_core_web_sm")

# Stopword set
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    """Preprocess text: tokenize, lowercase, remove stopwords, and lemmatize."""
    # Tokenize and lowercase
    tokens = word_tokenize(str(text).lower())

    # Remove stopwords and non-alphanumeric tokens
    filtered_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]

    # Lemmatize using spacy
    doc = nlp(" ".join(filtered_tokens))
    lemmatized_tokens = [token.lemma_ for token in doc]

    return " ".join(lemmatized_tokens)

# Load dataset
vuln_data = pd.read_csv(file_path)

# Preprocess title and description
vuln_data['title_preprocessed'] = vuln_data['title'].fillna("").apply(preprocess_text)
vuln_data['description_preprocessed'] = vuln_data['description'].fillna("").apply(preprocess_text)

# Save preprocessed data
vuln_data.to_csv("vuln_data_preprocessed.csv", index=False)
print("Preprocessing completed and saved to 'vuln_data_preprocessed.csv'")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Preprocessing completed and saved to 'vuln_data_preprocessed.csv'


In [4]:
endpoint_groups = vuln_data.groupby('endpoint')

In [5]:
from sentence_transformers import SentenceTransformer, util

# Load pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
# Severity weights mapping
severity_weights = {
    "low": 1,
    "medium": 2,
    "high": 3,
    "critical": 4
}

def calculate_severity_similarity(severity1: str, severity2:str):
    """Calculate similarity score based on severity."""
    if pd.isnull(severity1) or pd.isnull(severity2):
        return 0
    weight1 = severity_weights.get(severity1, 0)
    weight2 = severity_weights.get(severity2, 0)
    max_diff = max(severity_weights.values()) - min(severity_weights.values())
    similarity = 1 - (abs(int(weight1) - int(weight2)) / max_diff)
    return -1 + 2*similarity

def calculate_combined_similarity(title_desc_sim, severity_sim, severity_weight=0.1):
    combined_sim = title_desc_sim + (severity_weight * severity_sim)
    return max(0, min(combined_sim, 1))  # Ensure the result stays between 0 and 1

In [8]:
all_similarity_results = []

for endpoint, group in endpoint_groups:
    print(f"Processing endpoint: {endpoint}")
    group = group.reset_index(drop=True)

    # Step 1: Enforce CVE-based similarity
    cve_similarity_pairs = []
    cve_groups = group.groupby('cve')
    for cve_id, cve_group in cve_groups:
        if pd.notnull(cve_id):
            ids = cve_group['id'].tolist()
            for i in range(len(ids)):
                for j in range(i + 1, len(ids)):
                    cve_similarity_pairs.append({
                        'endpoint': endpoint,
                        'id1': ids[i],
                        'id2': ids[j],
                        'title_desc_similarity': 1.0,  # Maximum similarity for identical CVEs
                        'severity_similarity': 1.0,
                        'combined_similarity': 1.0,  # Identical CVEs are fully similar
                        'reason': 'Same CVE'
                    })

    # Step 2: Compute semantic similarity for all pairs
    embeddings = []
    for title, description in zip(group['title_preprocessed'], group['description_preprocessed']):
        text = title + description
        embeddings.append(model.encode(text, convert_to_tensor=True))
    # embeddings = group['title_preprocessed'].concat( group['description_preprocessed']).apply(lambda x: model.encode(x, convert_to_tensor=True)).tolist()
    for i in range(len(group)):
        for j in range(i + 1, len(group)):
            # Compute title/description similarity
            title_desc_similarity = util.cos_sim(embeddings[i], embeddings[j]).item()

            # Compute severity similarity
            severity_sim = calculate_severity_similarity(
                group.loc[i, 'severity'], group.loc[j, 'severity']
            )

            # Combine similarities
            combined_similarity = calculate_combined_similarity(title_desc_similarity, severity_sim)

            all_similarity_results.append({
                'endpoint': endpoint,
                'id1': group.loc[i, 'id'],
                'id2': group.loc[j, 'id'],
                'title_desc_similarity': title_desc_similarity,
                'severity_similarity': severity_sim,
                'combined_similarity': combined_similarity,
                'reason': 'Semantic + Severity Analysis'
            })


    # Merge CVE and semantic results
    all_similarity_results.extend(cve_similarity_pairs)



Processing endpoint: /apache
Processing endpoint: /api/encrypt
Processing endpoint: /assets
Processing endpoint: /cart
Processing endpoint: /comments
Processing endpoint: /config
Processing endpoint: /files
Processing endpoint: /headers
Processing endpoint: /login
Processing endpoint: /profile
Processing endpoint: /secret
Processing endpoint: /secure
Processing endpoint: /session
Processing endpoint: /subdomain
Processing endpoint: /upload


In [9]:
# Save similarity results
similarity_df = pd.DataFrame(all_similarity_results)
similarity_df.to_csv("vulnerability_similarity_scores.csv", index=False)
print("Similarity scores saved to 'vulnerability_similarity_scores.csv'")

Similarity scores saved to 'vulnerability_similarity_scores.csv'


In [10]:
from sklearn.cluster import DBSCAN
import numpy as np
import pandas as pd

# Create a similarity matrix for clustering
vuln_ids = pd.unique(similarity_df[['id1', 'id2']].values.ravel('K'))
id_to_index = {vuln_id: idx for idx, vuln_id in enumerate(vuln_ids)}
index_to_id = {idx: vuln_id for vuln_id, idx in id_to_index.items()}

# Create an adjacency matrix for similarity
n = len(vuln_ids)
similarity_matrix = np.zeros((n, n))

for _, row in similarity_df.iterrows():
    i, j = id_to_index[row['id1']], id_to_index[row['id2']]
    similarity_matrix[i, j] = row['combined_similarity']
    similarity_matrix[j, i] = row['combined_similarity']  # Symmetric

# Perform clustering using DBSCAN
dbscan = DBSCAN(eps=0.5, min_samples=1, metric="precomputed")
clusters = dbscan.fit_predict(1 - similarity_matrix)  # 1 - similarity for dissimilarity

# Add cluster tags to the original dataset
cluster_tags = {index_to_id[i]: f"group_{cluster}" for i, cluster in enumerate(clusters) if cluster != -1}
vuln_data['tag'] = vuln_data['id'].map(cluster_tags).fillna("unclustered")

# Save the tagged dataset
vuln_data.to_csv("vulnerability_with_tags.csv", index=False)
print("Tagged vulnerabilities saved to 'vulnerability_with_tags.csv'")


Tagged vulnerabilities saved to 'vulnerability_with_tags.csv'


In [ ]:
from flask import Flask, jsonify
import pandas as pd
from pyngrok import ngrok
import json

ngrok.set_auth_token("2rALs1L1VhMwt9but0MYQXXYFit_2rnSZuYUGd2p4ggy6A5HW")

# Convert the DataFrame to a JSON-compatible dictionary
def format_vulnerability_data(df):
    grouped_data = []
    for _, row in df.iterrows():
        grouped_data.append({
            "title": row.get("title", ""),
            "endpoint": row.get("endpoint", ""),
            "tag": row.get("tag", ""),
            "severity": row.get("severity", ""),
            "description": row.get("description", ""),
            "cve": row.get("cve", ""),
            "sensor": row.get("sensor", "")
        })
    return grouped_data

# Create Flask app
app = Flask(__name__)
app.json.sort_keys = False


# Define the /vulnerabilities endpoint
@app.route('/vulnerabilities', methods=['GET'])
def get_vulnerabilities():
    # Format data for the response
    formatted_data = format_vulnerability_data(vuln_data)
    return json.dumps(formatted_data, indent=4)

# Set up ngrok to expose the Flask app
public_url = ngrok.connect(5000)
print(f"Public URL: {public_url}")
app.run(port=5000)


Public URL: NgrokTunnel: "https://55d1-34-23-242-243.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [04/Jan/2025 16:32:51] "GET /vulnerabilities HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Jan/2025 16:32:52] "GET /favicon.ico HTTP/1.1" 404 -
